**Data loader**

In [ ]:
import torch
import torch.utils.data as data
import scipy.io as sio
import numpy as np
import os
import glob


class dataloader_light(data.Dataset):

    def __init__(self, filename):

        self.mat_data = sio.loadmat(filename)
        self.data = self.mat_data['data']
        self.labels = self.mat_data['label']

    def __getitem__(self, index):

        intensity = self.data[index].astype('double')
        label = self.labels[index]

        pair = {'intensity': intensity, 'label': label}
        return pair

    def __len__(self):
        return self.mat_data['data'].shape[0]


class dataloader_tmp(data.Dataset):

    def __init__(self, filename):

        self.mat_data = sio.loadmat(filename)
        self.data = self.mat_data['tmp_set']
        self.labels = self.mat_data['labels']

    def __getitem__(self, index):

        temp = self.data[index].astype('double')
        label = self.labels[index][0]

        pair = {'temp': temp, 'label': label}
        return pair

    def __len__(self):
        return self.mat_data['tmp_set'].shape[0]

**Model**

In [ ]:
import numpy as np
from numpy import asarray as ar
from numpy import sqrt
from numpy.random import rand, randn
import torch
import torch.nn as nn
from torch.autograd import Variable


class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        N = x.shape[0]
        return x.view(N, -1)


class Sandwich(nn.Module):
    def __init__(self, c_in, c_out, filter_size):
        super(Sandwich, self).__init__()

        self.net = nn.Sequential(
            nn.Conv1d(c_in, c_out, filter_size, stride=1, padding=(filter_size - 1) // 2),
            nn.BatchNorm1d(c_out),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.net(x)


class CNN_Light(nn.Module):
    def __init__(self, length, channel, num_layers, num_neu, pdrop):
        super(CNN_Light, self).__init__()

        self.len = length

        blocks = [Sandwich(1, channel, 7)]
        self.len = self.len // 2
        for _ in range(num_layers - 1):
            blocks.append(Sandwich(channel, channel, 3))
            self.len = self.len // 2

        blocks.append(Flatten())
        blocks.append(nn.Linear(self.len * channel, num_neu))
        blocks.append(nn.ReLU())
        blocks.append(nn.Dropout(p=pdrop))
        blocks.append(nn.Linear(num_neu, num_neu))
        blocks.append(nn.ReLU())
        blocks.append(nn.Dropout(p=pdrop))
        blocks.append(nn.Linear(num_neu, 1))

        self.net = nn.Sequential(*blocks)

    def forward(self, x):

        return self.net(x)


class CNN_Light_lite(nn.Module):
    def __init__(self, length, channel, num_layers, num_neu, pdrop):
        super(CNN_Light_lite, self).__init__()

        self.len = length

        blocks = [Sandwich(1, channel, 3)]
        self.len = self.len // 2
        for _ in range(num_layers - 1):
            blocks.append(Sandwich(channel, channel, 3))
            self.len = self.len // 2

        blocks.append(nn.Conv1d(channel, 1, 1, stride=1))

        self.net = nn.Sequential(*blocks)

    def forward(self, x):

        return torch.mean(self.net(x).squeeze(), dim=1, keepdim=True)


class CNN_Temp(nn.Module):
    def __init__(self, size, pdrop, num_neu):
        super(CNN_Temp, self).__init__()

        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=pdrop)

        self.fc1 = nn.Linear(size, num_neu)
        self.fc2 = nn.Linear(num_neu, num_neu)
        self.fc3 = nn.Linear(num_neu, num_neu)
        self.fc4 = nn.Linear(num_neu, 1)

    def forward(self, x):

        x = self.fc1(x)            # Din = 16*256, Dout = 1024
        x = self.relu(x)
        x = self.drop(x)
        x = self.fc2(x)            # Din = 16*256, Dout = 1024
        x = self.relu(x)
        x = self.drop(x)
        x = self.fc3(x)            # Din = 1024, Dout = 1024
        x = self.relu(x)
        x = self.drop(x)
        x = self.fc4(x)
        return x


class CNN_hybrid(nn.Module):
    def __init__(self, size, isdrop=0):
        super(CNN_hybrid, self).__init__()
        # Cin = 1, Cout = 256, Kernel_size = 11
        self.relu = nn.ReLU()
        self.isdrop = isdrop
        #self.maxpool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        #self.maxpool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        if self.isdrop == 1:
            self.drop = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):

        x = self.fc1(x)            # Din = 16*256, Dout = 1024
        x = self.relu(x)
        if self.isdrop == 1:
            x = self.drop(x)
        x = self.fc2(x)            # Din = 1024, Dout = 1024
        x = self.relu(x)
        if self.isdrop == 1:
            x = self.drop(x)
        x = self.fc3(x)

        return x


class CNN_large(nn.Module):
    def __init__(self, length, isdrop):
        super(CNN_large, self).__init__()
        # Cin = 1, Cout = 256, Kernel_size = 11
        self.isdrop = isdrop

        self.conv1 = nn.Conv1d(1, 64, 3, stride=1, padding=1)
        # Cin = 256, Cout = 256, Kernel_size = 5
        self.conv2 = nn.Conv1d(64, 128, 3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(128, 128, 3, stride=1, padding=1)

        # Batch Nromalization
        self.batnorm1 = nn.BatchNorm1d(64)
        self.batnorm2 = nn.BatchNorm1d(128)
        self.batnorm3 = nn.BatchNorm1d(128)

        self.relu = nn.ReLU()

        self.maxpool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.maxpool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        if self.isdrop == 1:
            self.drop = nn.Dropout(p=0.25)

        self.len = length

        self.fc1 = nn.Linear(int(self.len / 8) * 128, 128)

        #self.fc1 = nn.Linear(self.len, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):

        x = self.conv1(x)          # Cin = 1, Cout = 64, Kernel_size = 11
        x = self.batnorm1(x)
        x = self.relu(x)
        if self.isdrop == 1:
            x = self.drop(x)
        x = self.maxpool1(x)

        x = self.conv2(x)          # Cin = 64, Cout = 128, Kernel_size = 5
        x = self.batnorm2(x)
        x = self.relu(x)
        if self.isdrop == 1:
            x = self.drop(x)
        x = self.maxpool2(x)

        x = self.conv3(x)          # Cin = 64, Cout = 128, Kernel_size = 5
        x = self.batnorm3(x)
        x = self.relu(x)
        if self.isdrop == 1:
            x = self.drop(x)
        x = self.maxpool3(x)

        x = x.view(-1, int(self.len / 8) * 128)
        #x = x.squeeze(1)
        x = self.fc1(x)            # Din = 16*256, Dout = 1024
        x = self.relu(x)
        x = self.fc2(x)            # Din = 1024, Dout = 1024
        x = self.relu(x)
        x = self.fc3(x)            # Din = 1024, Dout = 1

        return x


In [ ]:
import numpy as np
import torch

import scipy.io as sio
import matplotlib.pyplot as plt

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

WINDOW_LIGHT = 8
WINDOW_TEMP = 16
LIGHT_RESO = 60
TEMP_RESO = 1
LENGTH_LIGHT = WINDOW_LIGHT * LIGHT_RESO
LENGTH_TEMP = WINDOW_TEMP * TEMP_RESO

gap = 4
is_month = False
light_net = CNN_Light(480, 128, 3, 128, 0).to(device)
light_net.load_state_dict(torch.load('light_net.w', map_location=device))
light_net.eval()
temp_net = CNN_Temp(LENGTH_TEMP * 2, 0.5, 256).to(device)
temp_net.load_state_dict(torch.load('temp_net_dropout_0.5.w', map_location=device))
temp_net.eval()


**Main**

In [ ]:
import numpy as np
import torch
import scipy.io as sio
import os
from easydict import EasyDict
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


def test(opt):

    temp_net = CNN_Temp(32, opt.dropout, opt.hidden).to(opt.device)

    net_path = os.path.join(opt.net_dir, opt.net_name)
    temp_net.load_state_dict(torch.load(net_path, map_location=opt.device))

    if not os.path.exists(opt.output_folder):
        os.makedirs(opt.output_folder)
    if not os.path.exists(opt.output_fig_folder):
        os.makedirs(opt.output_fig_folder)

    temp_net.eval()

    num = opt.num

    for n in range(num):

        data = sio.loadmat(os.path.join(opt.input_folder, str(n + 57) + '.mat'))
        temp_test = data['test_temp']

        results_temp = np.zeros((temp_test.shape[0], temp_test.shape[1]))

        for i in range(temp_test.shape[0]):
            temp = torch.from_numpy(temp_test[i, :, :]).float().to(opt.device)            
            temp_result = temp_net(temp)

            temp_result = torch.sigmoid(temp_result)

            results_temp[i, :] = temp_result.cpu().squeeze().detach().numpy()
    
        #fig_temp = os.path.join(opt.output_fig_folder, 'temp_'+str(n+1)+'.png')
        #plt.imshow(results_temp.transpose(), cmap='viridis', interpolation='nearest')
        #plt.colorbar()
        #plt.savefig(fig_temp)
        #plt.clf()

        path_temp = os.path.join(opt.output_folder, 'temp_' + str(n + 57) + '.mat')

        sio.savemat(path_temp, {'results': results_temp})
        print(n)


if __name__ == '__main__':
    opt = EasyDict()

    opt.dropout = 0
    opt.hidden = 256

    opt.net_dir = '/content/'
    opt.net_name = 'temp_net.w'

    opt.num = 1

    opt.output_folder = './results/Heatmaps_temp'
    opt.output_fig_folder = './results/figs_temp'
    opt.input_folder = '/content/'
    opt.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    test(opt)
